<a href="https://colab.research.google.com/github/shernee/CMPE256_SentimentAnalysis/blob/main/CodeBooks/Sentiment_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import gzip
import json
import string
import matplotlib.pyplot as plt
import pickle
import plotly.express as px
import numpy as np
from scipy import sparse

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import Perceptron, LogisticRegression, SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.svm import SVC

##Read review features pickle file and labeled products review data

In [ ]:
df_sentiment = pd.read_pickle('/content/drive/MyDrive/Sentiment_data/allproducts_labeled.pkl')
df_features = pd.read_pickle('/content/drive/MyDrive/Sentiment_data/all_products_review_features_df.pkl')

In [ ]:
df_features.head()

,review,review_length,token_count,stopword_count,punctuation_count,lemma_count,lemmas,expanded_lemmas
0,DOESN'T FIT MY LG 4,5,7,3,1,3,fit lg 4,not fit lg 4
1,Fits very nicely as long as you follow proper ...,12,13,5,1,7,fit nicely long follow proper instruction install,fit nicely long follow proper instruction install
2,It holds the phone very well enough.\nBut it's...,28,34,13,4,17,hold phone well enough arm pretty wobbly push ...,hold phone well enough arm pretty wobbly push ...
3,When you have an iPhone you usually go one of ...,241,302,143,39,120,iphone usually go one two way choose case eith...,iphone usually go one two way choose case eith...
4,This is my second Armorbox. I wore the first o...,43,48,19,4,25,second armorbox wore first one working industr...,second armorbox wore first one working industr...


In [ ]:
df_sentiment.head()

,review,lemmas,polarity,sentiment,expanded_lemmas
0,DOESN'T FIT MY LG 4,fit lg 4,"{'neg': 0.0, 'neu': 0.286, 'pos': 0.714, 'comp...",1,not fit lg 4
1,Fits very nicely as long as you follow proper ...,fit nicely long follow proper instruction install,"{'neg': 0.0, 'neu': 0.481, 'pos': 0.519, 'comp...",1,fit nicely long follow proper instruction install
2,It holds the phone very well enough.\nBut it's...,hold phone well enough arm pretty wobbly push ...,"{'neg': 0.094, 'neu': 0.657, 'pos': 0.249, 'co...",1,hold phone well enough arm pretty wobbly push ...
3,When you have an iPhone you usually go one of ...,iphone usually go one two way choose case eith...,"{'neg': 0.063, 'neu': 0.575, 'pos': 0.362, 'co...",1,iphone usually go one two way choose case eith...
4,This is my second Armorbox. I wore the first o...,second armorbox wore first one working industr...,"{'neg': 0.045, 'neu': 0.638, 'pos': 0.317, 'co...",1,second armorbox wore first one working industr...


##Convert the lemmas and labels into numpy vectors

In [ ]:
X = np.array(df_sentiment.loc[:, 'expanded_lemmas'])
Y = np.array(df_sentiment.loc[:, 'sentiment'])

##Train test split and TDIDF vectorizer

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,
                                   random_state=104, 
                                   test_size=0.33)

tv = TfidfVectorizer(min_df=0.0, max_df=1.0, ngram_range=(1,2),
                     sublinear_tf=True)
tv_train_features = tv.fit_transform(X_train)
tv_test_features = tv.transform(X_test)

In [ ]:
with open('/content/drive/MyDrive/Sentiment_data/tdidf.pkl', 'wb') as f:
  pickle.dump(tv, f)

##Doc2Vec embeddings with all data and then split into train and test .... not using

In [ ]:
# from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [ ]:
# # Make tagged documents for doc2vec
# tagged_X = [TaggedDocument(words=_d.split(" "), tags=[i]) for i, _d in enumerate(X)]

# # tagged_X_train = [TaggedDocument(words=_d.split(" "), tags=[i]) for i, _d in enumerate(X_train)]
# # tagged_X_test = [TaggedDocument(words=_d.split(" "), tags=[i]) for i, _d in enumerate(X_test)]


# # Make doc2vec model and save
# d2v_train_model = Doc2Vec(
#     dm=1,
#     vector_size=75,
#     window=3,
#     min_count=3,
#     alpha=0.03, 
#     min_alpha=0.0007, 
# )
# d2v_train_model.build_vocab(tagged_X)
# d2v_train_model.train(tagged_X, total_examples=d2v_train_model.corpus_count, epochs=30)
# # d2v_train_model.docvecs.vectors_docs

# d2v_train_model.save('/content/drive/MyDrive/Sentiment_data/d2v_model.d2v')

In [ ]:
# d2v_vectors = Doc2Vec.load('/content/drive/MyDrive/Sentiment_data/d2v_model.d2v')

In [ ]:
# X_train_docvecs, X_test_docvecs, Y_train, Y_test = train_test_split(d2v_vectors.docvecs.vectors_docs, Y,
#                                    random_state=104, 
#                                    test_size=0.33)

##SMOTE for balancing

In [ ]:
smote = SMOTE(sampling_strategy='minority')
X_train_resampled, Y_train_resampled = smote.fit_resample(X=tv_train_features, y=Y_train)

In [ ]:
print("Number of Negative reviews in the original dataset: ",np.count_nonzero(Y_train==0) / len(Y_train))
print("Number of Positive reviews in the original dataset: ",np.count_nonzero(Y_train==1) / len(Y_train))

print("Number of Negative reviews in the resampled dataset: ",np.count_nonzero(Y_train_resampled==0) / len(Y_train_resampled))
print("Number of Positive reviews in the resampled dataset: ",np.count_nonzero(Y_train_resampled==1) / len(Y_train_resampled))

## Tried Perceptron but not using it

In [ ]:
# perceptron_model = Perceptron(penalty='elasticnet', alpha=0.001)
# perceptron_model.fit(X_train_resampled, Y_train_resampled)

# with open('/content/drive/MyDrive/Sentiment_data/perceptron.pkl', 'wb') as f:
#   pickle.dump(perceptron_model, f)

In [ ]:
# with open('/content/drive/MyDrive/Sentiment_data/perceptron.pkl', 'rb') as f:
#   perceptron_model_tdidf_balanced = pickle.load(f)

# perceptron_predicted = perceptron_model.predict(tv_test_features)
# print(classification_report(y_true=Y_test,y_pred=perceptron_predicted))

## training the model using Logistic regression algorithm

In [ ]:
# LR on tdidf vectors without smote
lr_model_unbalanced = LogisticRegression(penalty='elasticnet', solver='saga', l1_ratio=0.5)
lr_model_unbalanced.fit(tv_train_features, Y_train)

with open('/content/drive/MyDrive/Sentiment_data/lr_unbalanced.pkl', 'wb') as f:
  pickle.dump(lr_model_unbalanced, f)

##generating the classification report for LR, our primary metric being Precision and then F1-score

In [ ]:
# with open('/content/drive/MyDrive/Sentiment_data/lr_unbalanced.pkl', 'rb') as f:
#   lr_model_tdidf_unbalanced = pickle.load(f)

lr_predicted_tdidf_unbalanced = lr_model_unbalanced.predict(tv_test_features)
print(classification_report(y_true=Y_test, y_pred=lr_predicted_tdidf_unbalanced))

              precision    recall  f1-score   support

           0       0.91      0.63      0.74     22930
           1       0.97      1.00      0.98    306926

    accuracy                           0.97    329856
   macro avg       0.94      0.81      0.86    329856
weighted avg       0.97      0.97      0.97    329856



In [ ]:
# LR on tdidf vectors after smote
lr_model_balanced = LogisticRegression(penalty='elasticnet', solver='saga', l1_ratio=0.5)
lr_model_balanced.fit(X_train_resampled, Y_train_resampled)

with open('/content/drive/MyDrive/Sentiment_data/lr_balanced.pkl', 'wb') as f:
  pickle.dump(lr_model_balanced, f)

In [ ]:
with open('/content/drive/MyDrive/Sentiment_data/lr_balanced.pkl', 'rb') as f:
  lr_model_tdidf_balanced = pickle.load(f)

lr_predicted_tdidf_balanced = lr_model_balanced.predict(tv_test_features)
print(classification_report(y_true=Y_test,y_pred=lr_predicted_tdidf_balanced))

              precision    recall  f1-score   support

           0       0.57      0.88      0.69     22930
           1       0.99      0.95      0.97    306926

    accuracy                           0.95    329856
   macro avg       0.78      0.92      0.83    329856
weighted avg       0.96      0.95      0.95    329856



In [ ]:
# # LR on docvec embeddings without smote
# lr_model_docvecs_unbalanced = LogisticRegression(penalty='none', solver='saga')
# lr_model_docvecs_unbalanced.fit(X_train_docvecs, Y_train)

# with open('/content/drive/MyDrive/Sentiment_data/lr_docvecs_unbalanced.pkl', 'wb') as f:
#   pickle.dump(lr_model_docvecs_unbalanced, f)

In [ ]:
# lr_predicted_docvecs_unbalanced = lr_model_docvecs_unbalanced.predict(X_test_docvecs)
# print(classification_report(y_true=Y_test, y_pred=lr_predicted_docvecs_unbalanced))

              precision    recall  f1-score   support

           0       0.48      0.13      0.20     22930
           1       0.94      0.99      0.96    306926

    accuracy                           0.93    329856
   macro avg       0.71      0.56      0.58    329856
weighted avg       0.91      0.93      0.91    329856



In [ ]:
# # LR on docvec embeddings after smote
# lr_model_docvecs_balanced = LogisticRegression(penalty='elasticnet', solver='saga', l1_ratio=0.5)
# lr_model_docvecs_balanced.fit(X_train_docvecs_resampled, Y_train_docvecs_resampled)

# with open('/content/drive/MyDrive/Sentiment_data/lr_docvecs_balanced.pkl', 'wb') as f:
#   pickle.dump(lr_model_docvecs_balanced, f)

In [ ]:
# lr_predicted_docvecs_balanced = lr_model_docvecs_balanced.predict(X_test_docvecs)
# print(classification_report(y_true=Y_test, y_pred=lr_predicted_docvecs_balanced))

              precision    recall  f1-score   support

           0       0.23      0.77      0.36     22930
           1       0.98      0.81      0.89    306926

    accuracy                           0.81    329856
   macro avg       0.61      0.79      0.62    329856
weighted avg       0.93      0.81      0.85    329856



##Trained model using SGD classifier and Multinomial Naive bayes, LR, Adaboost, Random forest, SVC to select best performing model , Chose LR model.

In [ ]:
sgd_model = SGDClassifier(loss='hinge', alpha=0.001)
sgd_model.fit(X_train_resampled, Y_train_resampled)

with open('/content/drive/MyDrive/Sentiment_data/sgd.pkl', 'wb') as f:
  pickle.dump(sgd_model, f)

In [ ]:
sgd_predicted = sgd_model.predict(tv_test_features)
print(classification_report(y_true=Y_test, y_pred=sgd_predicted))

Multinomial Naive Bayes

In [ ]:
mnb_model = MultinomialNB()
mnb_model.fit(X_train_resampled, Y_train_resampled)

with open('/content/drive/MyDrive/Sentiment_data/mnb.pkl', 'wb') as f:
  pickle.dump(mnb_model, f)

In [ ]:
mnb_predicted = mnb_model.predict(tv_test_features)
print(classification_report(y_true=Y_test, y_pred=mnb_predicted))

AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

sgd_ada_model = SGDClassifier(loss='hinge', alpha=0.001)

ab_model_tdidf_balanced = AdaBoostClassifier(n_estimators=50, base_estimator=sgd_ada_model, learning_rate=0.5, algorithm='SAMME')
ab_model_tdidf_balanced.fit(X_train_resampled, Y_train_resampled)

with open('/content/drive/MyDrive/Sentiment_data/ab_tdidf_balanced.pkl', 'wb') as f:
  pickle.dump(ab_model_tdidf_balanced, f)


In [ ]:
ab_predicted_tdidf_balanced = ab_model_tdidf_balanced.predict(tv_test_features)
print(classification_report(y_true=Y_test, y_pred=ab_predicted_tdidf_balanced))

              precision    recall  f1-score   support

           0       0.33      0.83      0.47     22930
           1       0.99      0.87      0.93    306926

    accuracy                           0.87    329856
   macro avg       0.66      0.85      0.70    329856
weighted avg       0.94      0.87      0.89    329856



Random forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_model_tdidf_balanced = RandomForestClassifier(n_estimators=50, min_samples_split=30, max_depth=150)
rf_model_tdidf_balanced.fit(X_train_resampled, Y_train_resampled)

with open('/content/drive/MyDrive/Sentiment_data/rf_tdidf_balanced.pkl', 'wb') as f:
  pickle.dump(rf_model_tdidf_balanced, f)

In [ ]:
rf_predicted_tdidf_balanced = rf_model_tdidf_balanced.predict(tv_test_features)
print(classification_report(y_true=Y_test, y_pred=rf_predicted_tdidf_balanced))

              precision    recall  f1-score   support

           0       0.53      0.57      0.55     22930
           1       0.97      0.96      0.96    306926

    accuracy                           0.93    329856
   macro avg       0.75      0.77      0.76    329856
weighted avg       0.94      0.93      0.94    329856



SVC

In [ ]:
svc_model_tdidf_balanced = SVC(C=0.01)
svc_model_tdidf_balanced.fit(X_train_resampled, Y_train_resampled)

with open('/content/drive/MyDrive/Sentiment_data/svc_tdidf_balanced.pkl', 'wb') as f:
  pickle.dump(svc_model_tdidf_balanced, f)

In [ ]:
svc_model_docvecs_balanced = SVC(C=0.01)
svc_model_docvecs_balanced.fit(X_train_docvecs_resampled, Y_train_docvecs_resampled)

with open('/content/drive/MyDrive/Sentiment_data/svc_docvecs_balanced.pkl', 'wb') as f:
  pickle.dump(svc_model_docvecs_balanced, f)

In [ ]:
svc_predicted_docvecs_balanced = svc_model_tdidf_balanced.predict(X_test_docvecs)
print(classification_report(y_true=Y_test, y_pred=svc_predicted_docvecs_balanced))

##end-to-end Pipeline 

Pipeline for input

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('vader_lexicon')
from nltk.tokenize import wordpunct_tokenize
from nltk.corpus import stopwords 
from nltk.stem import wordnet
from nltk.sentiment import SentimentIntensityAnalyzer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [ ]:
import contractions

In [ ]:
new_punctuation_list = list(string.punctuation) + ['...']
lem = wordnet.WordNetLemmatizer()

def lemmatize_input_review(review_text: str):
  expanded_review_text = contractions.fix("This isn't the best material")
  all_tokens = wordpunct_tokenize(expanded_review_text)
  tokens = []
  stopword_count = 0
  punctuation_count = 0

  for token in all_tokens:
    if token not in new_punctuation_list:
      cleaned_token = token.lower().replace(" ","")
      if cleaned_token not in new_stopwords_list:
        tokens.append(cleaned_token)
      else:
        stopword_count+=1
    else:
      punctuation_count+=1

  lemmatized_tokens = [lem.lemmatize(token) for token in tokens]
  lemma_str = " ".join(lemmatized_tokens)

  return lemma_str

def vectorize_input_lemma(lemma_string: str):
  with open('/content/drive/MyDrive/Sentiment_data/tdidf.pkl', 'rb') as f:
    tv = pickle.load(f)
  input_vector = tv.transform(np.array([lemma_string]))

  return input_vector

def classify_and_predict(input_vector):
  with open('/content/drive/MyDrive/Sentiment_data/lr_unbalanced.pkl', 'rb') as f:
    lr_model_tdidf_unbalanced = pickle.load(f)

  predicted_label = lr_model_tdidf_unbalanced.predict(input_vector)
  return predicted_label

In [ ]:
lemmas = lemmatize_input_review("This isn't the best material")
# lemma_vector = vectorize_input_lemma(lemmas)
# prediction = classify_and_predict(lemma_vector)

In [ ]:
['Positive' if p==1 else 'Negative' for p in prediction]

['Negative']

In [ ]:
lemmas

'isn best material'